In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
data_file = '/Users/nevinaresh/Documents/CPET Research/BxBData.json'
outcomes_file = '/Users/nevinaresh/Documents/CPET Research/CPX database_outcomes_meds_10_22.xlsx'

In [3]:
#reads in the data from the json file
raw_data = pd.read_json(data_file).transpose()
raw_data.head(2)

,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,TestDuration,...,P_Diast,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain
1,[Margaret],[Raffin],[MRN-03907458],[Female],[12/26/1946],[157.5],[45.36],[10/11/2018],[4:11 PM],[15:30],...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
2,[MYEDOE],[MYEDOE],[MRN-08/STANFORD CONCERT HF 08-7009-08],[Male],[5/1/1955],[175.3],[107.95],[6/19/2017],[2:34 PM],[26:23],...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22....","[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[634, 634, 634, 634, 634, 634, 634, 634, 634, ...","[756, 756, 756, 756, 756, 756, 756, 756, 756, ...","[1.73, 1.77, 1.61, 1.55, 0.84, 2.28, 1.5, 1.87...","[1.74, 2.26, 2.2, 2.13, 2.16, 1.69, 2.98, 2.28...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."


In [4]:
raw_data.shape

(4221, 54)

In [5]:
#formats columns and removes unnecessary
from src.jsonParsing import preprocess

preprocessed_data = preprocess(raw_data)
preprocessed_data.head(2)

,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test
0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,4:11 PM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00
1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,11:47 AM,...,"[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00


In [6]:
preprocessed_data.shape

(4220, 56)

In [7]:
#preprocessed_data[preprocessed_data['testDate'] == pd.to_datetime('10/11/2016', format='%m/%d/%Y')]

In [8]:
#filters those rows that have an ID1 of the form 'MRN-'MRNID
from src.jsonParsing import validmrn

data_with_mrn = validmrn(preprocessed_data)
data_with_mrn.head(2)

/Users/nevinaresh/Documents/Stanford/CPET-AI/src/jsonParsing.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_mrn['MRN'] = pd.to_numeric(valid_mrn['ID1'].str.split('-').str[1], errors='coerce')


,level_0,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,...,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test,MRN
0,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0
1,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0


In [9]:
data_with_mrn.shape

(3882, 58)

In [10]:
preprocessed_data[preprocessed_data['testDate'] == pd.to_datetime("10/11/2016", format='%m/%d/%Y')]

,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test
2555,2557,Brianne,Purcell,MRN-35543081,Female,1/14/1980,177.8,113.4,2016-10-11,2:44 PM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[25.4, 25.4, 25.4, 25.4, 25.4, 25.4, 25.4, 25....","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[645, 645, 645, 645, 645, 645, 645, 645, 645, ...","[763, 763, 763, 763, 763, 763, 763, 763, 763, ...","[2.1, 2.24, 1.66, 2.35, 2.8, 4.67, 1.37, 3.23,...","[2.92, 4.7, 2.96, 3.45, 3.48, 7.03, 3.23, 4.92...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2016-10-11 14:44:00
2659,2661,HML-PHX_102-006_JB,HML-PHX_102-006_JB,MRN-HML-PHX_102-006_JB,Male,1/28/1980,172.7,90.26,2016-10-11,11:19 AM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[647, 647, 647, 647, 647, 647, 647, 647, 647, ...","[764, 764, 764, 764, 764, 764, 764, 764, 764, ...","[1.28, 1.41, 1.13, 1.01, 1.29, 1.24, 1.06, 1.2...","[2.23, 2.12, 1.73, 1.97, 2.18, 2.06, 2.11, 1.8...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2016-10-11 11:19:00


In [11]:
outcomes_data = pd.read_excel(outcomes_file, sheet_name="Outcomes_updated")
outcomes_data.head(2)

,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_stroke_yrs,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs
0,1.0,4400,2016-04-29,4400-42489,0.0,NaT,NaN,NaN,NaT,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
1,2.0,84095,2012-09-07,84095-41159,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-03-31,1.56


In [12]:
outcomes_data[outcomes_data['Test Date'] == pd.to_datetime("10/11/2016", format='%m/%d/%Y')]

,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_stroke_yrs,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs
3789,2467.0,34568683,2016-10-11,34568683-42654,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,0.0,NaT,NaN,0.0,NaT,NaN,0.0,NaT,NaN
3903,2549.0,35543081,2016-10-11,35543081-42654,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,NaN,NaT,NaN,0.0,NaT,NaN,1.0,2016-10-12,0.0


In [13]:
from src.outcomeParsing import preprocess

death_data = preprocess(outcomes_data)
death_data.head(2)

,MRN,Test Date,Death,death_date
0,4400,2016-04-29,0.0,NaT
1,84095,2012-09-07,0.0,NaT


In [14]:
from src.outcomeParsing import mergeDataOutcome

#alternative outcomes where we have more data (death, cardiovascular related hospitalization)
data_and_outcomes = mergeDataOutcome(data_with_mrn, death_data, 365*7)
data_and_outcomes.head(2)

,level_0,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,...,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test,MRN,Test Date,Death,death_date
0,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0,2016-06-16,0,NaT
1,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0,2018-10-11,0,NaT


In [15]:
data_and_outcomes['Death'].value_counts()

Death
0    7824
1     202
Name: count, dtype: int64

In [16]:
x = np.array(data_and_outcomes['Time'][1]).astype(float)
y = np.array(data_and_outcomes['Rf'][1]).astype(float)
x_interp = np.arange(0, max(x), 5)
y_linear = interp1d(x, y, fill_value="extrapolate")
y_quadratic = interp1d(x, y, kind='quadratic', fill_value="extrapolate")
y_cubic = interp1d(x, y, kind='cubic', bounds_error=False, fill_value=y[[0]])
y_cubic(x_interp)
# plt.plot(x, y, 'o')
# plt.plot(x_interp, y_linear(x_interp), 'red')
# plt.plot(x_interp, y_quadratic(x_interp), 'green')
# plt.plot(x_interp, y_cubic(x_interp), 'yellow')
#plt.xticks(np.arange(0, max(data_and_outcomes['Time'][0]), 10))
#plt.yticks(np.arange(min(data_and_outcomes['Ti'][0]), max(data_and_outcomes['Ti'][0]), 10))

array([18.29      , 26.75180309, 17.6       , 18.19650866, 17.75      ,
       16.80733549, 14.95393743, 18.29      , 19.27504777, 17.80418492,
       18.23011889, 17.        , 18.67173638,  6.69331739, 29.73125515,
       21.77206923, 23.26      , 25.86      , 22.56      , 20.3198181 ,
       19.80994303, 19.23      , 17.51729583, 17.76540585, 31.91      ,
       28.3       , 24.37693586, 20.49948427, 21.06241421, 24.9       ,
       10.30441511, 30.46561596, 25.48237213, 23.67096542, 21.36049333,
       20.47595306, 24.43956075, 17.91679276, 19.60837243, 21.13      ,
       24.1       , 20.53445794, 21.92929974, 19.40278827, 15.76325571,
       11.9029391 , 31.17701181, 26.67      , 24.59      , 26.43      ,
       26.55      , 23.9       , 24.        , 20.76      , 26.09      ,
       26.70135764, 24.82112057, 25.68434787, 24.59      , 26.32      ,
       25.61052083, 24.8644637 , 25.49908778, 24.47407348, 24.69693036,
       24.06215745, 20.53657491, 27.33457433, 25.12055887, 26.79

In [17]:
type(data_and_outcomes['Time'][0])

list

In [18]:
list(filter(None, np.array(data_and_outcomes['P_Diast'][0])))

[]

In [19]:
features = data_and_outcomes.iloc[:,19:-5]
time = data_and_outcomes['Time']
#spline to normalize

In [20]:
features['Power']

0       [, , , , , , , , , , , , , , , , , , , , , , ,...
1       [, , , , , , , , , , , , , , , , , , , , , , ,...
2       [, , , , , , , , , , , , , , , , , , , , , , ,...
3       [, , , , , , , , , , , , , , , , , , , , , , ,...
4       [, , , , , , , , , , , , , , , , , , , , , , ,...
                              ...                        
8021    [, , , , , , , , , , , , , , , , , , , , , , ,...
8022    [, , , , , , , , , , , , , , , , , , , , , , ,...
8023    [, , , , , , , , , , , , , , , , , , , , , , ,...
8024    [, , , , , , , , , , , , , , , , , , , , , , ,...
8025    [, , , , , , , , , , , , , , , , , , , , , , ,...
Name: Power, Length: 8026, dtype: object

In [33]:
from src.preprocessing import feature_pruning
pruned_features = feature_pruning(features, time)
pruned_features

,Rf,VT,VE,VO2,VCO2,RQ,O2exp,CO2exp,VE/VO2,VE/VCO2,...,Grade,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,interpolated_time,time_start_at_zero
0,"[18.29, 26.75180308914143, 17.6, 18.1965086571...","[0.452, 0.6209065368829562, 0.549, 0.480135428...","[8.268, 16.642064796540417, 9.66, 8.7045635076...","[189.0858298, 559.7813181269902, 278.319911399...","[138.4675518, 406.25985713515706, 206.59725759...","[0.73, 0.7211426478591703, 0.74, 0.73839956118...","[82.3, 105.23671071014239, 95.99999999999999, ...","[9.5, 18.93893467019352, 14.699999999999998, 1...","[37.0, 26.217279142639605, 30.3, 31.4702629002...","[50.5, 36.14119972906257, 40.8, 42.73582409085...",...,"[0.0, 3.324374028376501e-52, -1.05057568862826...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.099999999999998, 22.1, 22.1000000000...","[52.0, 51.99999999999999, 51.99999999999999, 5...","[639.0, 639.0269907884979, 638.9999999999999, ...","[755.0, 755.0, 755.0, 755.0, 755.0, 755.0, 755...","[1.07, 0.8518315233722866, 1.1199999999999999,...","[2.21, 1.351790619599702, 2.29, 1.974617407259...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False
1,"[18.29, 26.75180308914143, 17.6, 18.1965086571...","[0.452, 0.6209065368829562, 0.549, 0.480135428...","[8.268, 16.642064796540417, 9.66, 8.7045635076...","[189.0858298, 559.7813181269902, 278.319911399...","[138.4675518, 406.25985713515706, 206.59725759...","[0.73, 0.7211426478591703, 0.74, 0.73839956118...","[82.3, 105.23671071014239, 95.99999999999999, ...","[9.5, 18.93893467019352, 14.699999999999998, 1...","[37.0, 26.217279142639605, 30.3, 31.4702629002...","[50.5, 36.14119972906257, 40.8, 42.73582409085...",...,"[0.0, 3.324374028376501e-52, -1.05057568862826...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.099999999999998, 22.1, 22.1000000000...","[52.0, 51.99999999999999, 51.99999999999999, 5...","[639.0, 639.0269907884979, 638.9999999999999, ...","[755.0, 755.0, 755.0, 755.0, 755.0, 755.0, 755...","[1.07, 0.8518315233722866, 1.1199999999999999,...","[2.21, 1.351790619599702, 2.29, 1.974617407259...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False
2,"[15.5, 14.809999999999999, 16.276026006845886,...","[0.974950132723634, 0.570080673841539, 0.38467...","[15.116, 8.446, 7.295059535106092, 14.38249432...","[497.4793992, 244.6065718, 249.22407477786766,...","[380.230353, 180.1116693, 168.75227230275695, ...","[0.76, 0.7400000000000001, 0.6815828542592595,...","[166.638966199834, 100.14655129023102, 65.4703...","[29.9827760481954, 14.8894057926413, 10.604808...","[28.2, 30.3, 25.17004030261357, 25.62370227604...","[36.9, 41.10000000000001, 36.73956952286325, 3...",...,"[0.0, 8.331010251020017e-92, 1.116051895098710...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.500000000000004, 22.5, 2...",NaN,"[649.0, 649.0, 649.0, 649.0000000000001, 649.0...","[760.0, 760.0, 760.0, 760.0000000000001, 760.0...","[1.45, 1.54, 1.5856177229154358, 1.83605408301...","[2.42, 2.5100000000000002, 1.972224460368583, ...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",True
3,"[15.5, 14.809999999999999, 16.276026006845886,...","[0.974950132723634, 0.570080673841539, 0.38467...","[15.116, 8.446, 7.295059535106092, 14.38249432...","[497.4793992, 244.6065718, 249.22407477786766,...","[380.230353, 180.1116693, 168.75227230275695, ...","[0.76, 0.7400000000000001, 0.6815828542592595,...","[166.638966199834, 100.14655129023102, 65.4703...","[29.9827760481954, 14.8894057926413, 10.604808...","[28.2, 30.3, 25.17004030261357, 25.62370227604...","[36.9, 41.10000000000001, 36.73956952286325, 3...",...,"[0.0, 8.331010251020017e-92, 1.116051895098710...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.500000000000004, 22.5, 2...",NaN,"[649.0, 649.0, 649.0, 649.0000000000001, 649.0...","[760.0, 760.0, 760.0, 760.0000000000001, 760.0...","[1.45, 1.54, 1.5856177229154358, 1.83605408301...","[2.42, 2.5100000000000002, 1.972224460368583, ...","[0, 

In [60]:
pruned_features['outcome'] = data_and_outcomes['Death']
pruned_features.head(2)

,Rf,VT,VE,VO2,VCO2,RQ,O2exp,CO2exp,VE/VO2,VE/VCO2,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,interpolated_time,time_start_at_zero,outcome
0,"[18.29, 26.75180308914143, 17.6, 18.1965086571...","[0.452, 0.6209065368829562, 0.549, 0.480135428...","[8.268, 16.642064796540417, 9.66, 8.7045635076...","[189.0858298, 559.7813181269902, 278.319911399...","[138.4675518, 406.25985713515706, 206.59725759...","[0.73, 0.7211426478591703, 0.74, 0.73839956118...","[82.3, 105.23671071014239, 95.99999999999999, ...","[9.5, 18.93893467019352, 14.699999999999998, 1...","[37.0, 26.217279142639605, 30.3, 31.4702629002...","[50.5, 36.14119972906257, 40.8, 42.73582409085...",...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.099999999999998, 22.1, 22.1000000000...","[52.0, 51.99999999999999, 51.99999999999999, 5...","[639.0, 639.0269907884979, 638.9999999999999, ...","[755.0, 755.0, 755.0, 755.0, 755.0, 755.0, 755...","[1.07, 0.8518315233722866, 1.1199999999999999,...","[2.21, 1.351790619599702, 2.29, 1.974617407259...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False,0
1,"[18.29, 26.75180308914143, 17.6, 18.1965086571...","[0.452, 0.6209065368829562, 0.549, 0.480135428...","[8.268, 16.642064796540417, 9.66, 8.7045635076...","[189.0858298, 559.7813181269902, 278.319911399...","[138.4675518, 406.25985713515706, 206.59725759...","[0.73, 0.7211426478591703, 0.74, 0.73839956118...","[82.3, 105.23671071014239, 95.99999999999999, ...","[9.5, 18.93893467019352, 14.699999999999998, 1...","[37.0, 26.217279142639605, 30.3, 31.4702629002...","[50.5, 36.14119972906257, 40.8, 42.73582409085...",...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.099999999999998, 22.1, 22.1000000000...","[52.0, 51.99999999999999, 51.99999999999999, 5...","[639.0, 639.0269907884979, 638.9999999999999, ...","[755.0, 755.0, 755.0, 755.0, 755.0, 755.0, 755...","[1.07, 0.8518315233722866, 1.1199999999999999,...","[2.21, 1.351790619599702, 2.29, 1.974617407259...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False,0


In [73]:
time_series_features = pd.DataFrame()
for col in pruned_features.iloc[:,0:-3]:
    for idx, row in pruned_features.iterrows():
        try:
            time_series_features.at[idx, col + '_mean'] = np.mean(row[col])
        except:
            time_series_features.at[idx, col + '_mean'] = np.nan
        try:
            time_series_features.at[idx, col + '_max'] = np.max(row[col])
        except:
            time_series_features.at[idx, col + '_max'] = np.nan
        try:
            time_series_features.at[idx, col + '_min'] = np.min(row[col])
        except:
            time_series_features.at[idx, col + '_min'] = np.nan
        try:
            time_series_features.at[idx, col + '_std'] = np.std(row[col])
        except:
            time_series_features.at[idx, col + '_std'] = np.nan
time_series_features

/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/480083839.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_series_features.at[idx, col + '_mean'] = np.mean(row[col])
/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/480083839.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_series_features.at[idx, col + '_max'] = np.max(row[col])
/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/480083839.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

,Rf_mean,Rf_max,Rf_min,Rf_std,VT_mean,VT_max,VT_min,VT_std,VE_mean,VE_max,...,PB_min,PB_std,Ti_mean,Ti_max,Ti_min,Ti_std,Te_mean,Te_max,Te_min,Te_std
0,26.343108,50.043451,6.693317,7.409566,0.817306,1.412597,0.212274,0.229081,21.862376,53.535141,...,755.0,9.504415e-14,0.995927,1.650559,0.137207,0.219165,1.472634,5.120000,0.297896,0.688078
1,26.343108,50.043451,6.693317,7.409566,0.817306,1.412597,0.212274,0.229081,21.862376,53.535141,...,755.0,9.504415e-14,0.995927,1.650559,0.137207,0.219165,1.472634,5.120000,0.297896,0.688078
2,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,760.0,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687
3,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,760.0,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687
4,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,760.0,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8022,19.402535,47.240000,-10.887953,9.491448,1.713780,5.233529,-0.478015,0.983272,33.479192,124.992000,...,761.0,1.147679e-13,1.261722,5.210000,-0.296907,0.611369,2.887820,20.597104,-0.374348,2.831246
8023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8024,22.783616,47.620000,0.508465,8.307436,1.293571,2.752462,0.302960,0.505078,31.283518,91.491000,...,761.0,8.294139e-14,1.146402,2.545001,0.490448,0.371409,1.916890,7.405464,0.690000,1.070630


In [79]:
time_series_features['output'] = data_and_outcomes['Death']
time_series_features = time_series_features.dropna(thresh=20)
time_series_features

/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/3304637137.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_series_features['output'] = data_and_outcomes['Death']
/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/3304637137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_series_features['output'] = data_and_outcomes['Death']


,Rf_mean,Rf_max,Rf_min,Rf_std,VT_mean,VT_max,VT_min,VT_std,VE_mean,VE_max,...,PB_std,Ti_mean,Ti_max,Ti_min,Ti_std,Te_mean,Te_max,Te_min,Te_std,output
0,26.343108,50.043451,6.693317,7.409566,0.817306,1.412597,0.212274,0.229081,21.862376,53.535141,...,9.504415e-14,0.995927,1.650559,0.137207,0.219165,1.472634,5.120000,0.297896,0.688078,0
1,26.343108,50.043451,6.693317,7.409566,0.817306,1.412597,0.212274,0.229081,21.862376,53.535141,...,9.504415e-14,0.995927,1.650559,0.137207,0.219165,1.472634,5.120000,0.297896,0.688078,0
2,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687,0
3,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687,0
4,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8016,22.120486,53.100000,3.484399,8.736458,1.318394,2.554328,0.029755,0.458215,30.436016,80.018000,...,1.064166e-13,1.277455,5.971501,-0.259734,0.721944,1.971006,8.640904,0.580000,1.201051,0
8017,22.120486,53.100000,3.484399,8.736458,1.318394,2.554328,0.029755,0.458215,30.436016,80.018000,...,1.064166e-13,1.277455,5.971501,-0.259734,0.721944,1.971006,8.640904,0.580000,1.201051,0
8018,22.120486,53.100000,3.484399,8.736458,1.318394,2.554328,0.029755,0.458215,30.436016,80.018000,...,1.064166e-13,1.277455,5.971501,-0.259734,0.721944,1.971006,8.640904,0.580000,1.201051,0
8022,19.402535,47.240000,-10.887953,9.491448,1.713780,5.233529,-0.478015,0.983272,33.479192,124.992000,...,1.147679e-13,1.261722,5.210000,-0.296907,0.611369,2.887820,20.597104,-0.374348,2.831246,0


In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(time_series_features[time_series_features.columns.drop('output')].to_numpy(), time_series_features['output'].to_numpy(), test_size=0.2, random_state=1115)

In [104]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, confusion_matrix
from tqdm import tqdm
from xgboost import XGBClassifier

In [100]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

/opt/miniconda3/envs/cpetai/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:09:12] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [101]:
y_pred = model.predict(X_test)

In [106]:
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[1318,    1],
       [  17,   23]])

In [107]:
tn, fp, fn, tp = conf_mat.ravel()

# Calculate precision
precision = tp / (tp + fp)

# Calculate recall (sensitivity)
recall = tp / (tp + fn)

# Calculate specificity
specificity = tn / (tn + fp)

print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('Specificity: ' + str(specificity))

Precision: 0.9583333333333334
Recall: 0.575
Specificity: 0.9992418498862775


In [121]:
values = model.feature_importances_
top_k_features = pd.DataFrame()
feat_coef_dict = dict(zip(time_series_features.columns, values))
top_k = sorted(feat_coef_dict.items(), key=lambda x: abs(x[1]), reverse=True)[0:20]
top_k_features['feature'] = [i[0] for i in top_k]
top_k_features['importance'] = [i[1] for i in top_k]
top_k_features['abs_importance'] = top_k_features['importance'].abs()
top_k_features

,feature,importance,abs_importance
0,PB_mean,0.043256,0.043256
1,Grade_std,0.036000,0.036000
2,Grade_max,0.030670,0.030670
3,Te_mean,0.030354,0.030354
4,VO2_std,0.026343,0.026343
5,CO2exp_std,0.026006,0.026006
6,Rf_max,0.024765,0.024765
7,Analyzer_Pressure_max,0.019617,0.019617
8,Ti_mean,0.019439,0.019439
9,METS_max,0.018881,0.018881
